# UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS  
**FACULTAD DE CIENCIAS MATEMÁTICAS**  
**ESCUELA DE POSGRADO**  
**MAESTRÍA EN ESTADÍSTICA MATEMÁTICA**

---

# **PRÁCTICA CALIFICADA Nº01**

**Asignatura:** Seminario de Análisis Multivariante  
**Docente:** Kelva Llanos Miranda

**Alumno:** Enrique Palacios Andrade  
**Código:** 24147015

**Fecha de Entrega:**  
Viernes 20 de Junio de 2025

---

## UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS  
**FACULTAD DE CIENCIAS MATEMÁTICAS**  
**ESCUELA DE POSGRADO**  
**MAESTRÍA EN ESTADÍSTICA MATEMÁTICA**

### PRÁCTICA CALIFICADA Nº01

**Alumno:** Enrique Palacios

---

## Indicaciones

- El desarrollo de la Práctica Calificada será presentado hasta las 23:59 horas del viernes 20/06 en el Classroom, en la actividad creada como `PC1-SAM-2025.01`.
- Si van a usar un software estadístico de su preferencia, presentar el desarrollo (el resultado obtenido) en cada pregunta, así como la interpretación o desarrollo de la prueba de hipótesis (Planteamiento de hipótesis, Estadístico de hipótesis, P-value, Decisión y Conclusión).

---

## Pregunta 1

Una empresa desea evaluar si el tipo de capacitación recibida por sus empleados influye significativamente en su desempeño. Se seleccionó aleatoriamente una muestra de 200 empleados, divididos en tres grupos según el tipo de capacitación recibida:

- Grupo 1: Capacitación presencial  
- Grupo 2: Capacitación virtual  
- Grupo 3: Capacitación mixta (presencial + virtual)

A cada empleado se le midieron tres indicadores de desempeño al finalizar el trimestre:

- **Y1:** Puntaje en una evaluación de conocimientos técnicos  
- **Y2:** Índice de eficiencia en tareas asignadas  
- **Y3:** Nivel de cumplimiento de metas trimestrales

El objetivo del análisis es determinar si existen diferencias multivariadas significativas entre los tres grupos de capacitación en cuanto al desempeño medido por las tres variables dependientes.

**Indicaciones:**

- Verifica los supuestos multivariados.
- Realiza la prueba MANOVA.
- Interpreta los resultados multivariados y univariados (si corresponde).
- Concluye sobre el efecto del tipo de capacitación.

📂 Los datos se encuentran en el archivo: `DATOS.PC1.SAM.2025.01.xlsx`

---



# Análisis MANOVA del impacto de la capacitación en el desempeño

Este análisis tiene como objetivo determinar si existen **diferencias multivariadas significativas** en el desempeño de los empleados según el tipo de capacitación recibida: presencial, virtual o mixta. Las tres variables dependientes evaluadas son:

- **Y1**: Puntaje en evaluación de conocimientos técnicos  
- **Y2**: Índice de eficiencia en tareas asignadas  
- **Y3**: Nivel de cumplimiento de metas trimestrales

In [101]:
!pip install openpyxl
!pip install statsmodels
!pip install bioinfokit
!pip install bioinfokit pandas
import pandas as pd
import pingouin as pg
import pandas as pd
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from statsmodels.multivariate.manova import MANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [102]:
# Ruta del archivo (ajusta el nombre si es necesario)
file_path = 'DATOS.PC1.SAM.2025.01.xlsx'

# Leer el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del DataFrame
print(df.head())

     Grupo          Y1          Y2          Y3
0  Grupo 1   445947358   508469571  1000123571
1  Grupo 1  4395062626  5283540134  9979697885
2  Grupo 1  4456284326  5282615737  9886718434
3  Grupo 1  4975704775  5110869527   100260084
4  Grupo 1  5752750784  5091350048   101615052



### 1. Verificación de supuestos multivariados

### a) **Normalidad univariada aproximada por grupo**


In [103]:
for col in ['Y1', 'Y2', 'Y3']:
    for group in df['Grupo'].unique():
        data = df[df['Grupo'] == group][col]
        stat, p = shapiro(data)
        print(f"{col} - {group}: p-valor Shapiro-Wilk = {p:.4f}")

Y1 - Grupo 1: p-valor Shapiro-Wilk = 0.0000
Y1 - Grupo 2: p-valor Shapiro-Wilk = 0.0000
Y1 - Grupo 3: p-valor Shapiro-Wilk = 0.0000
Y2 - Grupo 1: p-valor Shapiro-Wilk = 0.0000
Y2 - Grupo 2: p-valor Shapiro-Wilk = 0.0000
Y2 - Grupo 3: p-valor Shapiro-Wilk = 0.0000
Y3 - Grupo 1: p-valor Shapiro-Wilk = 0.0000
Y3 - Grupo 2: p-valor Shapiro-Wilk = 0.0000
Y3 - Grupo 3: p-valor Shapiro-Wilk = 0.0000



> Si **p > 0.05**, no se rechaza la normalidad univariada.  
> Aunque MANOVA es robusta con n > 30 por grupo, esta prueba ayuda como diagnóstico preliminar.



Los resultados muestran los p-valores de la prueba de Shapiro-Wilk para cada variable (Y1, Y2, Y3) dentro de cada grupo (Grupo 1, Grupo 2, Grupo 3).Un p-valor bajo (típicamente < 0.05) sugiere que la distribución de los datos para esa combinación de variable y grupo no es normal. En este caso, todos los p-valores son 0.0000,lo que indica una clara violación del supuesto de normalidad univariada en todos los grupos y variables.


### b) **Homogeneidad de matrices de covarianza: Prueba de Box**


In [104]:
# Load Excel file
df = pd.read_excel("DATOS.PC1.SAM.2025.01.xlsx")

# Ensure 'Grupo' column is categorical
df['Grupo'] = df['Grupo'].astype('category')

# Execute Box's M-test correctly
resultado_box = pg.box_m(data=df, dvs=['Y1', 'Y2', 'Y3'], group='Grupo')

# Display result
print("Resultado de la prueba de Box de igualdad de matrices de covarianza:")
print(resultado_box)

Resultado de la prueba de Box de igualdad de matrices de covarianza:
           Chi2    df           pval  equal_cov
box  522.612463  12.0  3.396808e-104      False


> Si **p > 0.05**, se cumple la homogeneidad de covarianzas.

---

### b) Homogeneidad de matrices de covarianza: Prueba de Box

* **Resultado:** La prueba M de Box arrojó un estadístico Chi2 de 522.61, con 12.0 grados de libertad, y un p-valor extremadamente bajo de $$3.396808 \times 10^{-104}$$. La salida `equal_cov` es `False`.

* **Interpretación:** Dado que el p-valor ($$3.396808 \times 10^{-104}$$) es significativamente menor que el nivel de significancia común ($$\alpha = 0.05$$ e incluso $0.001$), se **rechaza la hipótesis nula de homogeneidad de las matrices de covarianza**. Esto implica que las matrices de covarianza de las variables dependientes (Y1, Y2, Y3) son significativamente diferentes entre los grupos de capacitación.

* **Implicación para MANOVA:** Esta violación del supuesto de homogeneidad de las matrices de covarianza es una consideración importante para la validez de los resultados del MANOVA. Cuando este supuesto no se cumple, el estadístico **Pillai's Trace** es generalmente el más robusto y, por lo tanto, el más recomendado para interpretar los resultados multivariados del MANOVA, ya que es menos sensible a las diferencias en las matrices de covarianza entre los grupos.


### 2. Prueba MANOVA

### Estandarización de variables y aplicación de MANOVA

In [105]:
df_scaled = df.copy()
df_scaled[['Y1_z', 'Y2_z', 'Y3_z']] = StandardScaler().fit_transform(df[['Y1', 'Y2', 'Y3']])

manova = MANOVA.from_formula("Y1_z + Y2_z + Y3_z ~ Grupo", data=df_scaled)
print(manova.mv_test())

                  Multivariate linear model
                                                             
-------------------------------------------------------------
       Intercept        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.6811 3.0000 193.0000 30.1271 0.0000
         Pillai's trace 0.3189 3.0000 193.0000 30.1271 0.0000
 Hotelling-Lawley trace 0.4683 3.0000 193.0000 30.1271 0.0000
    Roy's greatest root 0.4683 3.0000 193.0000 30.1271 0.0000
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
         Grupo          Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.5225 6.0000 386.0000 24.6665 0.0000
         Pillai's trace 0.5143 6.0000 388.0000 22.3861 0.0000
 Hotelling-Lawley trace 0.

> Se generan los estadísticos multivariados: Wilks’ Lambda, Pillai’s Trace, Hotelling’s Trace y Roy’s Root.


**Interpretación de los Resultados del MANOVA:**

El resultado de la prueba MANOVA se presenta en dos secciones principales: el "Intercept" y el factor "Grupo". La sección 'Intercept' evalúa el efecto del intercepto en el modelo, y aunque sus p-valores son 0.0000 (indicando significancia), no es el foco de la hipótesis de investigación.

La sección más relevante es la correspondiente al factor 'Grupo'. Aquí, todos los estadísticos multivariados (Wilks' lambda, Pillai's trace, Hotelling-Lawley trace y Roy's greatest root) muestran un p-valor de 0.0000. Dado que este p-valor es mucho menor que el nivel de significancia común (por ejemplo, $$\alpha = 0.05$$), se rechaza la hipótesis nula multivariada. Esto significa que existe una diferencia estadísticamente significativa en las medias conjuntas de las variables dependientes (Y1_z, Y2_z, Y3_z) entre al menos dos de los grupos definidos por la variable 'Grupo'. En otras palabras, el tipo de capacitación tiene un efecto significativo en el rendimiento general de los empleados (medido por las variables Y1, Y2 y Y3 combinadas). Es importante recordar los hallazgos de la prueba de Box (heterogeneidad de matrices de covarianza); en presencia de matrices de covarianza desiguales, el Pillai's Trace se considera el estadístico más robusto y conservador. Afortunadamente, en este caso, todos los estadísticos fueron altamente significativos, lo que refuerza la solidez de la conclusión sobre las diferencias multivariadas entre los grupos.


### 3. Interpretación de resultados

### a) **Multivariado**

- **Wilks’ Lambda = 0.5225, p &lt; 0.0001
- También significativos: Pillai, Hotelling, Roy

> Se rechaza la hipótesis nula: hay diferencias multivariadas significativas entre los grupos de capacitación. Esto significa que el tipo de capacitación tiene un impacto colectivo en el conjunto de variables de desempeño (Y1, Y2, Y3).


### b) **Univariado (ANOVA por variable)**

In [106]:
for y in ['Y1', 'Y2', 'Y3']:
    model = ols(f"{y} ~ Grupo", data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(f"\nANOVA para {y}")
    print(anova_table)


ANOVA para Y1
                sum_sq     df          F        PR(>F)
Grupo     1.621286e+20    2.0  26.086431  9.136880e-11
Residual  6.059677e+20  195.0        NaN           NaN

ANOVA para Y2
                sum_sq     df          F    PR(>F)
Grupo     4.391581e+19    2.0  10.834538  0.000035
Residual  3.951983e+20  195.0        NaN       NaN

ANOVA para Y3
                sum_sq     df         F        PR(>F)
Grupo     6.553067e+20    2.0  47.46751  1.599775e-17
Residual  1.346024e+21  195.0       NaN           NaN


**Resultados del ANOVA Univariado:**

ANOVA para Y1: F = 26.09, p-valor = $$9.14 \times 10^{-11}$$
ANOVA para Y2: F = 10.83, p-valor = $$0.000035$$
ANOVA para Y3: F = 47.47, p-valor = $$1.60 \times 10^{-17}$$
> Todas las variables (Y1, Y2, Y3) muestran p-valores individuales significativamente por debajo de 0.05. Esto indica que cada una de ellas contribuye de manera significativa a las diferencias observadas entre los grupos, lo que complementa el resultado multivariado. En otras palabras, el tipo de capacitación tiene un efecto individual y significativo en el desempeño de cada una de las métricas evaluadas.






## 4. Conclusión final

El análisis MANOVA indica un efecto estadísticamente significativo del tipo de capacitación en el desempeño de los empleados, al considerar conjuntamente las variables Y1, Y2 y Y3. Esto confirma que la modalidad de formación impacta el rendimiento general.

Los resultados univariados refuerzan esta conclusión, mostrando que las diferencias no solo son colectivas, sino que cada variable de desempeño (Y1, Y2, Y3) también es influenciada significativamente por el tipo de capacitación. Por lo tanto, la modalidad de capacitación (presencial, virtual o mixta) influye de forma significativa y medible en el desempeño organizacional en sus diversas dimensiones.

Los hallazgos sugieren enfáticamente la necesidad de evaluar y adaptar los planes de formación a la modalidad que maximice el impacto positivo en las métricas de desempeño. Esto se debe al efecto comprobado del tipo de capacitación en el rendimiento de los empleados.

---



## **Pregunta 2**

Sea $\mathbf{X} = (X_1, X_2, X_3)'$ un vector aleatorio con distribución normal de media

$$
\mu = \begin{pmatrix} 2 \\ 0 \\ 1 \end{pmatrix}, \quad
\Sigma = \begin{pmatrix}
4 & 1 & 2 \\
1 & 3 & 0 \\
2 & 0 & 5
\end{pmatrix}
$$

**a)** Encontrar la ley de la variable aleatoria $Y = 2X_1 - X_2 + X_3$  
**b)** Encontrar un vector $\mathbf{a} \in \mathbb{R}^{2 \times 1}$ tal que la variable aleatoria  
$X_1 - \mathbf{a}'(X_2, X_3)'$ sea independiente de $(X_2, X_3)'$  
**c)** Calcular la distribución de $X_1$ condicionada a $(X_2, X_3)' = (x_2^0, x_3^0)'$

---




## **Desarrollo Pregunta 2 – Distribución Normal Multivariada**

---

### Contexto del Problema

Sea $\mathbf{X}$ un vector aleatorio que sigue una distribución normal multivariada con las siguientes características:

$$\mathbf{X} = \begin{pmatrix} X_1 \\ X_2 \\ X_3 \end{pmatrix} \sim N\left( \mu = \begin{pmatrix} 2 \\ 0 \\ 1 \end{pmatrix}, \Sigma = \begin{pmatrix} 4 & 1 & 2 \\ 1 & 3 & 0 \\ 2 & 0 & 5 \end{pmatrix} \right)$$

Aquí, $\mu$ es el **vector de medias** y $\Sigma$ es la **matriz de covarianza**.

---

### a) Ley de $Y = 2X_1 - X_2 + X_3$

Dado que $Y$ es una combinación lineal de variables con distribución normal, $Y$ también seguirá una **distribución normal**. Para definirla, necesitamos su media y varianza.

#### **Media de $Y$ ($\mu_Y$):**
La esperanza de una combinación lineal es la combinación lineal de las esperanzas.
$$\mu_Y = E[Y] = E[2X_1 - X_2 + X_3]$$
Aplicando la linealidad de la esperanza:
$$\mu_Y = 2 \cdot E[X_1] - E[X_2] + E[X_3]$$
Sustituimos las medias de $\mu$:
$$\mu_Y = 2 \cdot 2 - 0 + 1 = 4 - 0 + 1 = 5$$

#### **Varianza de $Y$ ($\sigma_Y^2$):**
La varianza de una combinación lineal de variables de un vector normal multivariado se calcula como $\mathbf{a}' \Sigma \mathbf{a}$, donde $\mathbf{a}$ es el vector de coeficientes de la combinación lineal.
Para $Y = 2X_1 - 1X_2 + 1X_3$, el vector de coeficientes es $\mathbf{a} = \begin{pmatrix} 2 \\ -1 \\ 1 \end{pmatrix}$.
La matriz de covarianza $\Sigma$ es $\begin{pmatrix} 4 & 1 & 2 \\ 1 & 3 & 0 \\ 2 & 0 & 5 \end{pmatrix}$.

**Paso 1: Calcular $\Sigma \mathbf{a}$**
$$\Sigma \mathbf{a} = \begin{pmatrix} 4 & 1 & 2 \\ 1 & 3 & 0 \\ 2 & 0 & 5 \end{pmatrix} \begin{pmatrix} 2 \\ -1 \\ 1 \end{pmatrix} = \begin{pmatrix} (4)(2) + (1)(-1) + (2)(1) \\ (1)(2) + (3)(-1) + (0)(1) \\ (2)(2) + (0)(-1) + (5)(1) \end{pmatrix} = \begin{pmatrix} 8 - 1 + 2 \\ 2 - 3 + 0 \\ 4 + 0 + 5 \end{pmatrix} = \begin{pmatrix} 9 \\ -1 \\ 9 \end{pmatrix}$$

**Paso 2: Calcular $\mathbf{a}' (\Sigma \mathbf{a})$**
$$\sigma_Y^2 = \begin{pmatrix} 2 & -1 & 1 \end{pmatrix} \begin{pmatrix} 9 \\ -1 \\ 9 \end{pmatrix} = (2)(9) + (-1)(-1) + (1)(9) = 18 + 1 + 9 = 28$$

**Resultado:**
La variable aleatoria $Y$ sigue una distribución normal con media 5 y varianza 28.
$$Y \sim \mathcal{N}(5, 28)$$

---

### b) Encontrar $\mathbf{a}$ para Independencia

Buscamos un vector $\mathbf{a}$ tal que $X_1 - \mathbf{a}'(X_2, X_3)'$ sea independiente de $(X_2, X_3)'$. Para una distribución normal multivariada, esto se cumple cuando:

$$\mathbf{a}' = \Sigma_{12} \Sigma_{22}^{-1}$$

Primero, particionamos la matriz de covarianza $\Sigma$:
$$\Sigma = \begin{pmatrix} \Sigma_{11} & \Sigma_{12} \\ \Sigma_{21} & \Sigma_{22} \end{pmatrix}$$

De la matriz de covarianza original:
$$\Sigma = \begin{pmatrix} 4 & 1 & 2 \\ 1 & 3 & 0 \\ 2 & 0 & 5 \end{pmatrix}$$
Identificamos las submatrices:
* $\Sigma_{12}$ (covarianza de $X_1$ con $X_2$ y $X_3$) = $\begin{pmatrix} 1 & 2 \end{pmatrix}$
* $\Sigma_{22}$ (matriz de covarianza de $X_2$ y $X_3$) = $\begin{pmatrix} 3 & 0 \\ 0 & 5 \end{pmatrix}$

**Paso 1: Calcular la inversa de $\Sigma_{22}$**
Para una matriz diagonal, la inversa se obtiene invirtiendo cada elemento de la diagonal:
$$\Sigma_{22}^{-1} = \begin{pmatrix} 1/3 & 0 \\ 0 & 1/5 \end{pmatrix}$$

**Paso 2: Calcular $\mathbf{a}' = \Sigma_{12} \Sigma_{22}^{-1}$**
$$\mathbf{a}' = \begin{pmatrix} 1 & 2 \end{pmatrix} \begin{pmatrix} 1/3 & 0 \\ 0 & 1/5 \end{pmatrix} = \begin{pmatrix} (1)(1/3) + (2)(0) & (1)(0) + (2)(1/5) \end{pmatrix}$$
$$\mathbf{a}' = \begin{pmatrix} 1/3 & 2/5 \end{pmatrix}$$

**Resultado:**
El vector $\mathbf{a}$ es $\begin{pmatrix} 1/3 \\ 2/5 \end{pmatrix}$, y su transpuesta es $\mathbf{a}' = \left( \frac{1}{3}, \frac{2}{5} \right)$.

---

### c) Distribución Condicional de $X_1 | (X_2, X_3) = (x_2^0, x_3^0)$

La distribución condicional de una variable de un vector normal multivariado, dadas las otras variables, es también una **distribución normal**. Necesitamos su media y varianza condicional.

Recordemos las particiones de $\mu$ y $\Sigma$:
* $\mu_1 = 2$ (media de $X_1$)
* $\mu_{23} = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$ (medias de $X_2$ y $X_3$)
* $\mathbf{x}_{23}^0 = \begin{pmatrix} x_2^0 \\ x_3^0 \end{pmatrix}$ (valores observados de $X_2$ y $X_3$)
* $\Sigma_{11} = 4$ (varianza de $X_1$)
* $\Sigma_{12} = \begin{pmatrix} 1 & 2 \end{pmatrix}$
* $\Sigma_{21} = \begin{pmatrix} 1 \\ 2 \end{pmatrix}$ (transpuesta de $\Sigma_{12}$)
* $\Sigma_{22}^{-1} = \begin{pmatrix} 1/3 & 0 \\ 0 & 1/5 \end{pmatrix}$

#### **Media Condicional ($\mathbb{E}[X_1 \mid X_2, X_3]$):**
La fórmula para la media condicional es:
$$\mathbb{E}[X_1 \mid \mathbf{X}_{23}] = \mu_1 + \Sigma_{12} \Sigma_{22}^{-1} (\mathbf{X}_{23} - \mu_{23})$$
Ya sabemos que $\Sigma_{12} \Sigma_{22}^{-1} = \left( \frac{1}{3}, \frac{2}{5} \right)$.
Sustituimos los valores:
$$\mathbb{E}[X_1 \mid X_2, X_3] = 2 + \left( \frac{1}{3}, \frac{2}{5} \right) \left( \begin{pmatrix} x_2^0 \\ x_3^0 \end{pmatrix} - \begin{pmatrix} 0 \\ 1 \end{pmatrix} \right)$$
$$\mathbb{E}[X_1 \mid X_2, X_3] = 2 + \left( \frac{1}{3}, \frac{2}{5} \right) \begin{pmatrix} x_2^0 \\ x_3^0 - 1 \end{pmatrix}$$
$$\mathbb{E}[X_1 \mid X_2, X_3] = 2 + \frac{1}{3}x_2^0 + \frac{2}{5}(x_3^0 - 1)$$
$$\mathbb{E}[X_1 \mid X_2, X_3] = 2 + \frac{1}{3}x_2^0 + \frac{2}{5}x_3^0 - \frac{2}{5}$$
Combinamos las constantes:
$$\mathbb{E}[X_1 \mid X_2, X_3] = \left( 2 - \frac{2}{5} \right) + \frac{1}{3}x_2^0 + \frac{2}{5}x_3^0 = \left( \frac{10 - 2}{5} \right) + \frac{1}{3}x_2^0 + \frac{2}{5}x_3^0$$
$$\mathbb{E}[X_1 \mid X_2, X_3] = \frac{8}{5} + \frac{1}{3}x_2^0 + \frac{2}{5}x_3^0$$

#### **Varianza Condicional ($\text{Var}(X_1 \mid X_2, X_3)$):**
La varianza condicional se calcula como:
$$\text{Var}(X_1 \mid X_2, X_3) = \Sigma_{11} - \Sigma_{12} \Sigma_{22}^{-1} \Sigma_{21}$$

**Paso 1: Calcular $\Sigma_{12} \Sigma_{22}^{-1} \Sigma_{21}$**
Ya sabemos que $\Sigma_{12} \Sigma_{22}^{-1} = \left( \frac{1}{3}, \frac{2}{5} \right)$.
$$\left( \frac{1}{3}, \frac{2}{5} \right) \begin{pmatrix} 1 \\ 2 \end{pmatrix} = \left( \frac{1}{3} \cdot 1 \right) + \left( \frac{2}{5} \cdot 2 \right) = \frac{1}{3} + \frac{4}{5}$$
Para sumar estas fracciones, usamos el denominador común 15:
$$\frac{1}{3} + \frac{4}{5} = \frac{5}{15} + \frac{12}{15} = \frac{17}{15}$$

**Paso 2: Restar este resultado de $\Sigma_{11}$**
$$\text{Var}(X_1 \mid X_2, X_3) = 4 - \frac{17}{15}$$
Para restar, convertimos 4 a una fracción con denominador 15:
$$\text{Var}(X_1 \mid X_2, X_3) = \frac{4 \cdot 15}{15} - \frac{17}{15} = \frac{60}{15} - \frac{17}{15} = \frac{43}{15}$$
El valor decimal aproximado es $\approx 2.867$.

**Resultado:**
La distribución condicional de $X_1$ dado $(X_2, X_3) = (x_2^0, x_3^0)$ es una distribución normal con la media y varianza calculadas:
$$X_1 \mid (X_2=x_2^0, X_3=x_3^0) \sim \mathcal{N}\left( \frac{8}{5} + \frac{1}{3}x_2^0 + \frac{2}{5}x_3^0, \frac{43}{15} \right)$$

## **Pregunta 3**

Se generan dos muestras aleatorias de tamaño $n_1 = 120$ y $n_2 = 140$, correspondientes a trabajadores de jornada completa y jornada parcial, respectivamente. Se asume que las muestras provienen de distribuciones normales trivariadas:

$$
X_1 \sim N_3(\mu_1, \Sigma), \quad X_2 \sim N_3(\mu_2, \Sigma)
$$

Con:

$$
\mu_1 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}, \quad
\mu_2 = \begin{pmatrix} 72 \\ 63 \\ 79 \end{pmatrix}, \quad
\Sigma = \begin{pmatrix}
16 & 4 & 2 \\
4 & 9 & 3 \\
2 & 3 & 25
\end{pmatrix}
$$

**a)** Obtener los vectores de medias y matrices de covarianzas muestrales.  
**b)** Contrastar la hipótesis $H_0: \mu_1 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$ vs. $H_1: \mu_1 \neq \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$  
**c)** Contrastar $H_0: \mu_1 = \mu_2$ vs. $H_1: \mu_1 \neq \mu_2$ mediante:  

- Prueba $T^2$ de Hotelling

---

## **Desarrollo Pregunta 3 – Comparación de Dos Grupos Multivariados**

---

## Contexto del Problema

Se tienen dos muestras independientes, que representan dos grupos distintos, ambos siguiendo una distribución normal multivariada de dimensión 3.

* **Muestra 1 (Jornada Completa):**
    * Tamaño: $n_1 = 120$
    * Distribución: $X_1 \sim \mathcal{N}_3(\mu_1, \Sigma)$

* **Muestra 2 (Jornada Parcial):**
    * Tamaño: $n_2 = 140$
    * Distribución: $X_2 \sim \mathcal{N}_3(\mu_2, \Sigma)$

Las medias y la matriz de covarianza poblacionales son:
$$\mu_1 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}, \quad \mu_2 = \begin{pmatrix} 72 \\ 63 \\ 79 \end{pmatrix}, \quad \Sigma = \begin{pmatrix} 16 & 4 & 2 \\ 4 & 9 & 3 \\ 2 & 3 & 25 \end{pmatrix}$$

---

### a) Medias y Matrices de Covarianzas Muestrales

En este problema, se nos proporcionan directamente los valores **poblacionales** de las medias ($\mu_1, \mu_2$) y la matriz de covarianza común ($\Sigma$). Dado que estamos trabajando con estas características poblacionales, y en ausencia de datos muestrales específicos para calcular estimaciones, **asumimos que las medias muestrales y las matrices de covarianza muestrales son iguales a sus contrapartes poblacionales**. Esta es una simplificación común en problemas teóricos cuando los parámetros poblacionales son conocidos.

* **Medias Muestrales:**
    * Para la Muestra 1: $\bar{X}_1 = \mu_1 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$
    * Para la Muestra 2: $\bar{X}_2 = \mu_2 = \begin{pmatrix} 72 \\ 63 \\ 79 \end{pmatrix}$

* **Matrices de Covarianzas Muestrales:**
    * Dado que la matriz de covarianza poblacional $\Sigma$ es común para ambos grupos, las matrices de covarianza muestrales (si se asumieran iguales a la poblacional) serían:
        * $S_1 = \Sigma = \begin{pmatrix} 16 & 4 & 2 \\ 4 & 9 & 3 \\ 2 & 3 & 25 \end{pmatrix}$
        * $S_2 = \Sigma = \begin{pmatrix} 16 & 4 & 2 \\ 4 & 9 & 3 \\ 2 & 3 & 25 \end{pmatrix}$
    * Para pruebas de dos muestras con covarianza común, se usa una matriz de covarianza conjunta o "pooled" ($S_p$), que en este caso, al ser conocida $\Sigma$, será $S_p = \Sigma$.

---

### b) Contraste de Hipótesis $H_0: \mu_1 = (70, 65, 80)'$

Este contraste evalúa si la media poblacional del grupo 1 es igual a un vector de valores específicos ($\mu_0$). Se utiliza el estadístico $T^2$ de Hotelling para una muestra.

* **Hipótesis Nula ($H_0$):** $\mu_1 = \mu_0 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$
* **Hipótesis Alternativa ($H_1$):** $\mu_1 \neq \mu_0$

El estadístico $T^2$ para una muestra se calcula como:
$$T^2 = n_1 (\bar{X}_1 - \mu_0)' \Sigma^{-1} (\bar{X}_1 - \mu_0)$$

**Paso 1: Calcular la diferencia $(\bar{X}_1 - \mu_0)$**
Según la información proporcionada en el contexto del problema y la parte (a), asumimos que $\bar{X}_1$ es igual a la media poblacional $\mu_1$.
$$\bar{X}_1 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$$
La media teórica bajo la hipótesis nula es:
$$\mu_0 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix}$$
Entonces, la diferencia es:
$$(\bar{X}_1 - \mu_0) = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix} - \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}$$

**Paso 2: Calcular el valor del estadístico $T^2$**
Sustituimos la diferencia calculada en la fórmula del $T^2$:
$$T^2 = n_1 (\bar{X}_1 - \mu_0)' \Sigma^{-1} (\bar{X}_1 - \mu_0)$$
$$T^2 = 120 \cdot \begin{pmatrix} 0 & 0 & 0 \end{pmatrix} \Sigma^{-1} \begin{pmatrix} 0 \\ 0 \\ 0 \end{pmatrix}$$
Cualquier vector nulo multiplicado por una matriz y luego por su transpuesta resultará en 0.
$$T^2 = 120 \cdot 0 = 0$$

**Conclusión:**
Como $T^2 = 0$, que es el valor más bajo posible, indica que no hay ninguna discrepancia entre la media muestral observada (que es igual a la media poblacional proporcionada y la media teórica bajo la hipótesis nula). Un $T^2$ de 0 siempre lleva a la no-rechazo de la hipótesis nula. Por lo tanto, **no se rechaza la hipótesis nula ($H_0$)**, lo que significa que la media poblacional del Grupo 1 es consistente con el vector $(70, 65, 80)'$.

---

### c) Contraste $H_0: \mu_1 = \mu_2$ usando Hotelling $T^2$

Este contraste evalúa si las medias poblacionales de los dos grupos son iguales. Se utiliza el estadístico $T^2$ de Hotelling para dos muestras.

* **Hipótesis Nula ($H_0$):** $\mu_1 = \mu_2$ (no hay diferencia entre las medias multivariadas)
* **Hipótesis Alternativa ($H_1$):** $\mu_1 \neq \mu_2$ (hay diferencia entre las medias multivariadas)

#### **Paso 1: Calcular la diferencia de medias $(\bar{X}_1 - \bar{X}_2)$**
Asumiendo $\bar{X}_1 = \mu_1$ y $\bar{X}_2 = \mu_2$:
$$\bar{X}_1 - \bar{X}_2 = \begin{pmatrix} 70 \\ 65 \\ 80 \end{pmatrix} - \begin{pmatrix} 72 \\ 63 \\ 79 \end{pmatrix} = \begin{pmatrix} 70 - 72 \\ 65 - 63 \\ 80 - 79 \end{pmatrix} = \begin{pmatrix} -2 \\ 2 \\ 1 \end{pmatrix}$$

#### **Paso 2: Identificar la matriz de covarianza común ($S_p$ o $\Sigma$)**
La fórmula general para dos muestras es $T^2 = \frac{n_1 n_2}{n_1 + n_2} (\bar{X}_1 - \bar{X}_2)' S_p^{-1} (\bar{X}_1 - \bar{X}_2)$.
Dado que en el contexto del problema se nos proporciona una matriz de covarianza poblacional $\Sigma$ que es **común** para ambos grupos, $S_p$ (la matriz de covarianza pooled o combinada) se asume directamente como $\Sigma$.
$$S_p = \Sigma = \begin{pmatrix} 16 & 4 & 2 \\ 4 & 9 & 3 \\ 2 & 3 & 25 \end{pmatrix}$$

#### **Paso 3: Calcular la inversa de la matriz de covarianza común ($\Sigma^{-1}$)**

Para calcular la inversa de una matriz de 3x3, se puede usar la fórmula general $\Sigma^{-1} = \frac{1}{\text{det}(\Sigma)} \text{adj}(\Sigma)$, donde $\text{adj}(\Sigma)$ es la adjunta de $\Sigma$.

**Paso 3a: Calcular el determinante de $\Sigma$**
$$\text{det}(\Sigma) = \begin{vmatrix} 16 & 4 & 2 \\ 4 & 9 & 3 \\ 2 & 3 & 25 \end{vmatrix}$$
Usando la expansión por cofactores a lo largo de la primera fila:
$$\text{det}(\Sigma) = 16 \cdot \text{det}\begin{pmatrix} 9 & 3 \\ 3 & 25 \end{pmatrix} - 4 \cdot \text{det}\begin{pmatrix} 4 & 3 \\ 2 & 25 \end{pmatrix} + 2 \cdot \text{det}\begin{pmatrix} 4 & 9 \\ 2 & 3 \end{pmatrix}$$
$$= 16 \cdot ((9)(25) - (3)(3)) - 4 \cdot ((4)(25) - (3)(2)) + 2 \cdot ((4)(3) - (9)(2))$$
$$= 16 \cdot (225 - 9) - 4 \cdot (100 - 6) + 2 \cdot (12 - 18)$$
$$= 16 \cdot (216) - 4 \cdot (94) + 2 \cdot (-6)$$
$$= 3456 - 376 - 12$$
$$\text{det}(\Sigma) = 3068$$

**Paso 3b: Calcular la matriz de cofactores de $\Sigma$**
La matriz de cofactores $C$ donde $C_{ij} = (-1)^{i+j} M_{ij}$ (siendo $M_{ij}$ el menor complementario).

$C_{11} = (9)(25) - (3)(3) = 225 - 9 = 216$
$C_{12} = -((4)(25) - (3)(2)) = -(100 - 6) = -94$
$C_{13} = (4)(3) - (9)(2) = 12 - 18 = -6$

$C_{21} = -((4)(25) - (2)(3)) = -(100 - 6) = -94$
$C_{22} = (16)(25) - (2)(2) = 400 - 4 = 396$
$C_{23} = -((16)(3) - (4)(2)) = -(48 - 8) = -40$

$C_{31} = ((4)(3) - (2)(9)) = 12 - 18 = -6$
$C_{32} = -((16)(3) - (2)(4)) = -(48 - 8) = -40$
$C_{33} = ((16)(9) - (4)(4)) = 144 - 16 = 128$

La matriz de cofactores es:
$$C = \begin{pmatrix} 216 & -94 & -6 \\ -94 & 396 & -40 \\ -6 & -40 & 128 \end{pmatrix}$$

**Paso 3c: Calcular la adjunta de $\Sigma$ ($\text{adj}(\Sigma)$)**
La adjunta es la transpuesta de la matriz de cofactores:
$$\text{adj}(\Sigma) = C^T = \begin{pmatrix} 216 & -94 & -6 \\ -94 & 396 & -40 \\ -6 & -40 & 128 \end{pmatrix}$$
(En este caso, la matriz de cofactores es simétrica, por lo que $C^T = C$)

**Paso 3d: Calcular la inversa $\Sigma^{-1}$**
$$\Sigma^{-1} = \frac{1}{\text{det}(\Sigma)} \text{adj}(\Sigma) = \frac{1}{3068} \begin{pmatrix} 216 & -94 & -6 \\ -94 & 396 & -40 \\ -6 & -40 & 128 \end{pmatrix}$$
Podemos dejarla en esta forma o calcular los valores decimales:
$$\Sigma^{-1} \approx \begin{pmatrix} 0.0704 & -0.0306 & -0.0020 \\ -0.0306 & 0.1291 & -0.0130 \\ -0.0020 & -0.0130 & 0.0417 \end{pmatrix}$$

#### **Paso 4: Calcular el producto $(\bar{X}_1 - \bar{X}_2)' \Sigma^{-1}$**
Sabemos $(\bar{X}_1 - \bar{X}_2)' = \begin{pmatrix} -2 & 2 & 1 \end{pmatrix}$.
$$(\bar{X}_1 - \bar{X}_2)' \Sigma^{-1} = \begin{pmatrix} -2 & 2 & 1 \end{pmatrix} \frac{1}{3068} \begin{pmatrix} 216 & -94 & -6 \\ -94 & 396 & -40 \\ -6 & -40 & 128 \end{pmatrix}$$
Multiplicamos el vector fila por la matriz (sin el escalar $1/3068$ por ahora):
$$= \frac{1}{3068} \begin{pmatrix} (-2)(216) + (2)(-94) + (1)(-6) & (-2)(-94) + (2)(396) + (1)(-40) & (-2)(-6) + (2)(-40) + (1)(128) \end{pmatrix}$$
$$= \frac{1}{3068} \begin{pmatrix} -432 - 188 - 6 & 188 + 792 - 40 & 12 - 80 + 128 \end{pmatrix}$$
$$= \frac{1}{3068} \begin{pmatrix} -626 & 940 & 60 \end{pmatrix}$$

#### **Paso 5: Calcular el producto total $(\bar{X}_1 - \bar{X}_2)' \Sigma^{-1} (\bar{X}_1 - \bar{X}_2)$**
Multiplicamos el resultado del Paso 4 por el vector columna $(\bar{X}_1 - \bar{X}_2)$:
$$= \frac{1}{3068} \begin{pmatrix} -626 & 940 & 60 \end{pmatrix} \begin{pmatrix} -2 \\ 2 \\ 1 \end{pmatrix}$$
$$= \frac{1}{3068} [(-626)(-2) + (940)(2) + (60)(1)]$$
$$= \frac{1}{3068} [1252 + 1880 + 60]$$
$$= \frac{3192}{3068}$$
$$ \approx 1.040417$$

#### **Paso 6: Calcular el factor $\frac{n_1 n_2}{n_1 + n_2}$**
$$n_1 = 120, \quad n_2 = 140$$
$$\frac{n_1 n_2}{n_1 + n_2} = \frac{120 \cdot 140}{120 + 140} = \frac{16800}{260} = \frac{1680}{26} = \frac{840}{13} \approx 64.61538$$

#### **Paso 7: Calcular el estadístico $T^2$ final**
$$T^2 = \left( \frac{n_1 n_2}{n_1 + n_2} \right) (\bar{X}_1 - \bar{X}_2)' \Sigma^{-1} (\bar{X}_1 - \bar{X}_2)$$
$$T^2 = \frac{840}{13} \cdot \frac{3192}{3068}$$
$$T^2 = \frac{2681280}{39884}$$
$$T^2 \approx 67.234$$

**Resultado del Estadístico de Prueba:**
El valor del estadístico $T^2$ de Hotelling es $T^2 \approx 67.234$.

**Para la interpretación (no solicitada explícitamente, pero útil):**
Este valor de $T^2$ se compara con una distribución $F$.
El estadístico $T^2$ se relaciona con una distribución $F$ de la siguiente manera:
$$F = \frac{n_1+n_2-p-1}{(n_1+n_2-2)p} T^2 \quad \text{o en el caso de covarianza conocida } \quad F = \frac{T^2}{p}$$
Para un test de Hotelling $T^2$ con $\Sigma$ conocida, el estadístico $T^2$ tiene una distribución Chi-cuadrado con $p$ grados de libertad ($\chi^2_p$). En este caso, $p=3$.

Si $T^2 \sim \chi^2_p$ (exactamente bajo condiciones ideales de $\Sigma$ conocida y muestras grandes, o si se considera la distribución asintótica), se compararía $67.234$ con un valor crítico de $\chi^2_3$ a un nivel de significancia $\alpha$. Un valor tan grande como $67.234$ para una $\chi^2_3$ (donde el percentil 99.9 es ~16.27) indica fuertemente que se rechazaría la hipótesis nula.

**Conclusión (basada en el valor calculado):**
Un valor de $T^2$ de aproximadamente $67.234$ es considerablemente grande. Esto sugiere que hay una **diferencia estadísticamente significativa** entre las medias de los dos grupos, lo que llevaría a **rechazar la hipótesis nula ($H_0: \mu_1 = \mu_2$)**.

In [107]:
import numpy as np

# Definición de las diferencias de medias y la matriz de covarianza
mu_diff = np.array([-2, 2, 1])
Sigma = np.array([[16, 4, 2],
                  [4, 9, 3],
                  [2, 3, 25]])

# Calcular la inversa de Sigma
inv_Sigma = np.linalg.inv(Sigma)

# Tamaños de las muestras
n1 = 120
n2 = 140

# Calcular el estadístico T^2
T2 = (n1 * n2) / (n1 + n2) * mu_diff @ inv_Sigma @ mu_diff
print(f"El estadístico T^2 es: {T2:.4f}")

El estadístico T^2 es: 67.2270
